### Imports

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname('__file__'), '..')))

In [2]:
# %pip install -r ../requirements.txt

In [3]:
from src.encoding.encoders import *
from src.encoding.tools import create_encoding_pipeline
from src.models.sklearn_api_model import save_object, Model
from src.models.sklearn_api_models_config import get_model
from src.datasets.base_tabular_dataset import BaseTabularDataset
from src.experiments.base_experiment import BaseExperiment
import src.features as ft
import logging
import pandas as pd
import pathlib
import numpy as np

In [4]:
import logging
import os
import sys
import datetime as dt
from typing import List, Union, Optional
import pathlib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from src.datasets.base_tabular_dataset import BaseTabularDataset
from src.encoding.tools import create_encoding_pipeline
from src.experiments.features_selection import get_features, explore_features
from src.models.sklearn_api_model import Model, ModelTree
import src.features as ft
import mlflow.sklearn
import mlflow
import mlflow.data.pandas_dataset
from mlflow.models import infer_signature
import os
import matplotlib.pyplot as plt
%matplotlib widget
import cudf as cd
import numpy as np
import re


### Config

In [5]:
# Define a logger used by all modules
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO, encoding="utf-8",
                    format="%(name)s %(asctime)s: %(levelname)s: %(message)s", handlers=[logging.StreamHandler()])

In [6]:
# Define the root directory of the project
root_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
root_dir = pathlib.Path(root_dir)

##### Encoding Pipeline

In [7]:
# Define an encoding scheme to create the encoding pipeline
encoders_dict = {
    'number': {
        'as_number': {
            'imputers': [imputers.SimpleImputer(strategy='mean')],
            'encoders': [
                ne.StandardScaler(),
            ]
        }
    },
    'category': {
        'as_category': {
            'imputers': [imputers.SimpleImputer(strategy='most_frequent')],
            'encoders': [
                ne.MultiTargetEncoder(drop_invariant=True, return_df=True),
            ]
        }
    },
    'datetime': {
        'as_number': {
            'imputers': [de.DateFeatureExtractor()],
            'encoders': [
                ne.CyclicalFeatures(drop_original=True)
            ]
        },
        'as_category': {
            'imputers': [de.DateFeatureExtractor(dtype='category')],
            'encoders': [
                ne.MultiTargetEncoder(drop_invariant=True, return_df=True),
            ]
        }
    },
    'boolean': {
        'as_number': {
            'imputers': [imputers.SimpleImputer(strategy='most_frequent')],
             'encoders': [ne.BooleanEncoder()]
         }

    }
}

In [8]:
# Create the encoding pipeline
pipeline = create_encoding_pipeline(encoders_dict)

Creating encoding pipeline


In [9]:
pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('featureunion-1',
                                                  FeatureUnion(transformer_list=[('simpleimputer',
                                                                                  SimpleImputer())])),
                                                 ('featureunion-2',
                                                  FeatureUnion(transformer_list=[('standardscaler',
                                                                                  StandardScaler())]))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x75a915f3f700>),
                                ('pipe...
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x75a915f3f2b0>),
                                ('pipeline-5',
                                 Pipeline(steps=[('featureunion-1',
                                                  FeatureUnion(transformer_list=[('simpleimputer',
                                                                                  SimpleImputer(strategy='most_frequent'))])),
                                                 ('featureunion-2',
                                                  FeatureUnion(transformer_list=[('booleanencoder',
                                                                                  BooleanEncoder())]))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x75a915f3ee60>)],
                  verbose=True)

##### Dataset

In [10]:
# Define the configuration for the fetching of the data
fetch_config = {
    "data_start": '01-01-2019',
    "data_stop": '31-12-2023',
    'data_dir': root_dir / 'data',
    "locations": ['CHU Dijon', 'CH Beaune', 'CH Semur', 'CH Chatillon Montbard', 'CH privé Dijon', 'CH Langres', 'CH Chaumont', 'HNFC', 'CHU Besançon']
    }

In [11]:
# Select the features to be used in the dataset
ars_features_class = [
    ft.HospitalFeatures,
    ft.AirQualityFeatures,
    ft.EpidemiologicalFeatures,
    # ft.FireFightersFeatures(include_calls=False),
    ft.GoogleTrendFeatures,
    ft.MeteorologicalFeatures,
    ft.SociologicalFeatures,
    ft.SportsCompetitionFeatures,
    ft.TrafficFeatures
    ]

In [12]:
# Select the target columns to be predicted
target_colomns = ['nb_emmergencies']
# target_colomns = ['nb_vers_hospit']
# target_colomns = ['nb_hospit_np_adults%%J+1%%mean_7J']

In [13]:
# Define the splitting scheme to create the sets
split_config = {'test_size': 0.2, 'val_size': 0.2, 'shuffle': False}

In [14]:
# Create the dataset and fetch the data from the source then call get_dataset() method to fill the different attributes (X and y) of the different sets, and their encodings
arsTabularDataset = BaseTabularDataset(features_classes=ars_features_class, logger=logger, fetch_config=fetch_config)

root 2024-11-15 16:31:48,330: INFO: Initialisation de la classe BaseTabularDataset
root 2024-11-15 16:31:48,335: INFO: Initialisation des features
root 2024-11-15 16:31:48,336: INFO: Fetching dataset
root 2024-11-15 16:31:48,550: INFO: hospitalfeatures's data already fetched for CHU Dijon
root 2024-11-15 16:31:49,771: INFO: hospitalfeatures's data already fetched for CH Beaune
root 2024-11-15 16:31:50,283: INFO: hospitalfeatures's data already fetched for CH Semur
root 2024-11-15 16:31:50,535: INFO: hospitalfeatures's data already fetched for CH Chatillon Montbard
root 2024-11-15 16:31:50,785: INFO: hospitalfeatures's data already fetched for CH privé Dijon
root 2024-11-15 16:31:51,036: INFO: hospitalfeatures's data already fetched for CH Langres
root 2024-11-15 16:31:51,328: INFO: hospitalfeatures's data already fetched for CH Chaumont
root 2024-11-15 16:31:51,903: INFO: hospitalfeatures's data already fetched for HNFC
root 2024-11-15 16:31:52,139: INFO: hospitalfeatures's data alread

In [15]:
for location in fetch_config['locations']:
    # Define the configuration of the dataset
    dataset_config = {
    'from_date': '01-01-2019',
    'to_date': '30-12-2023',
    'locations': [location],
    # 'locations': ['CHU Dijon', 'CH Beaune', 'CH Semur', 'CH Chatillon Montbard', 'CH privé Dijon'],
    # 'axis': 'rows',
    'shift': range(1, 8, 1),
    # 'rolling_window': [7, 14, 31, 365],
    'freq': '1D',
    'split_config': split_config,
    'create_X_y': True,
    'encoding_pipeline': pipeline,
    'targets_names': target_colomns,
    'targets_shift': -3,
    'targets_rolling_window': 3,
    'targets_history_shifts': range(1, 8, 1),
    'targets_history_rolling_windows': [7, 14, 31, 365],
    'targets_locations': [location],
    'drop_constant_thr': 1.0,
    'data_dir': root_dir / 'data'
    }

    arsTabularDataset.get_dataset(**dataset_config)

    df = arsTabularDataset.enc_data

    # Identifie les colonnes contenant des NaN
    cols_with_nan = df.columns[df.isna().any()].tolist()
    print("Colonnes contenant des NaN:", cols_with_nan)

    # Affiche la liste des index des lignes contenant des NaN pour chaque colonne
    nan_indices = {col: df[df[col].isna()].index.tolist() for col in cols_with_nan}
    print(nan_indices)

    df.to_csv(fetch_config['data_dir'] / f'datasets/full_dataset_{location}.csv')
    df_target = arsTabularDataset.data[arsTabularDataset.targets_names]
    df_target.to_csv(fetch_config['data_dir'] / f'datasets/full_dataset_{location}_targets.csv')

root 2024-11-15 16:32:09,876: INFO: Getting the dataset from 01-01-2019 to 30-12-2023 for CHU Dijon
root 2024-11-15 16:32:10,107: INFO: Getting data for hospitalfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:10,114: INFO: Augmentation des features...
root 2024-11-15 16:32:10,121: INFO: Getting data for airqualityfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:10,130: INFO: Augmentation des features...
root 2024-11-15 16:32:10,158: INFO: Getting data for epidemiologicalfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:10,163: INFO: Augmentation des features...
root 2024-11-15 16:32:10,174: INFO: Getting data for googletrendfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:10,202: INFO: Augmentation des features...
root 2024-11-15 16:32:10,353: INFO: Getting data for meteorologicalfeatures from

            nb_emmergencies  nb_emmergencies%%J-1  nb_emmergencies%%J-2  \
date                                                                      
2019-01-01              202                 202.0                 202.0   
2019-01-02              198                 202.0                 202.0   
2019-01-03              186                 198.0                 202.0   
2019-01-04              186                 186.0                 198.0   
2019-01-05              204                 186.0                 186.0   
...                     ...                   ...                   ...   
2023-12-26              177                 187.0                 177.0   
2023-12-27              165                 177.0                 187.0   
2023-12-28              192                 165.0                 177.0   
2023-12-29              157                 192.0                 165.0   
2023-12-30              212                 157.0                 192.0   

            nb_emmergenc

root 2024-11-15 16:32:10,892: INFO: 30 features not encoded (same unit as target)


[ColumnTransformer] .... (4 of 4) Processing pipeline-5, total=   0.0s
Colonnes contenant des NaN: []
{}


root 2024-11-15 16:32:12,016: INFO: Getting the dataset from 01-01-2019 to 30-12-2023 for CH Beaune
root 2024-11-15 16:32:12,255: INFO: Getting data for hospitalfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:12,263: INFO: Augmentation des features...
root 2024-11-15 16:32:12,272: INFO: Getting data for airqualityfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:12,282: INFO: Augmentation des features...
root 2024-11-15 16:32:12,310: INFO: Getting data for epidemiologicalfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:12,315: INFO: Augmentation des features...
root 2024-11-15 16:32:12,326: INFO: Getting data for googletrendfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:12,357: INFO: Augmentation des features...
root 2024-11-15 16:32:12,502: INFO: Getting data for meteorologicalfeatures from

...

[ColumnTransformer] .... (1 of 4) Processing pipeline-1, total=   0.0s
[ColumnTransformer] .... (2 of 4) Processing pipeline-3, total=   0.0s
[ColumnTransformer] .... (3 of 4) Processing pipeline-4, total=   0.0s
[ColumnTransformer] .... (4 of 4) Processing pipeline-5, total=   0.0s
Colonnes contenant des NaN: []
{}


root 2024-11-15 16:32:26,240: INFO: Getting the dataset from 01-01-2019 to 30-12-2023 for CHU Besançon
root 2024-11-15 16:32:26,477: INFO: Getting data for hospitalfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:26,484: INFO: Augmentation des features...
root 2024-11-15 16:32:26,492: INFO: Getting data for airqualityfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:26,525: INFO: Augmentation des features...
root 2024-11-15 16:32:26,576: INFO: Getting data for epidemiologicalfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:26,582: INFO: Augmentation des features...
root 2024-11-15 16:32:26,593: INFO: Getting data for googletrendfeatures from 2019-01-01 00:00:00 to 2023-12-30 00:00:00, at a 1D frequency
root 2024-11-15 16:32:26,621: INFO: Augmentation des features...
root 2024-11-15 16:32:26,781: INFO: Getting data for meteorologicalfeatures f

Dropped columns with zero variance: ['meteo_snow']
            nb_emmergencies  nb_emmergencies%%J-1  nb_emmergencies%%J-2  \
date                                                                      
2019-01-01              186                 186.0                 186.0   
2019-01-02              228                 186.0                 186.0   
2019-01-03              183                 228.0                 186.0   
2019-01-04              212                 183.0                 228.0   
2019-01-05              188                 212.0                 183.0   
...                     ...                   ...                   ...   
2023-12-26              224                 152.0                 174.0   
2023-12-27              181                 224.0                 152.0   
2023-12-28              201                 181.0                 224.0   
2023-12-29              218                 201.0                 181.0   
2023-12-30              210                 218.0

root 2024-11-15 16:32:27,196: INFO: 30 features not encoded (same unit as target)


Dropped 40 constant columns from both sets: {'PM10_FR26010', 'PM25_FR26094', 'PM10_FR26094', 'NO2_FR26094', 'NO2_FR26010'}
X shape: (1162, 745), y shape: (1162, 1)
[ColumnTransformer] .... (1 of 4) Processing pipeline-1, total=   0.0s
[ColumnTransformer] .... (2 of 4) Processing pipeline-3, total=   0.0s
[ColumnTransformer] .... (3 of 4) Processing pipeline-4, total=   0.0s
[ColumnTransformer] .... (4 of 4) Processing pipeline-5, total=   0.0s
Colonnes contenant des NaN: []
{}
